## Lab One: Exploring Table Data

#### By: Harrison Noble and Henry Lambson 

### 1. Business Understanding

According to UCI [Ref 1], this dataset contains "red and white wine varients of the Portuguese Vinho Verde wine". Two datasets were collected, one for the reds and one for the whites.

The data was gathered via physical/chemical tests (such as acid content) and judgements of the quality. The value for the quality was determined by having three wine testers try each wine and score it from 0 to 10, with 0 being very bad and 10 being excellent. The median of the three scores was used in the final dataset.

The wine quality dataset contains 1599 red wine samples and 4898 white wine samples, with 6497 samples in total. There are 12 unique features including the quality value. Each feature is aside from the quality is numerical, with the quality being categorical.     

Through analyzing this dataset, we will be able to help predict which wines should continue to be tested in order to eventually reach the market. If we were to design the algorithm for high quality wine testers, we would only want to send them the samples that are deemed to be a 7 or higher on the quality by the algorithm. In order to accomplish this, we will want to reduce the amount of false-negatives in order to ensure that no top quality wine is passed up on. To this end, some underperforming wines will pass through as false-positives. We accept this tradeoff so long that the algorithm narrows down the samples by 60%, assuming the majority of the wines are rated below 7. This could be useful for wine testers as it cuts down the total amount of samples that would need to be tested.

Additionally, this dataset could be used to predict how the chemical or physical compenents of the wine correspond to the quality. By exploring the relationship between the features provided and the quality of the wine, we will be able to determine which feature values or ratios correspond to higher qualities. Wine producers would be interested in this result as they could determine which features and how much of them lead to higher quality wines. For this model to work well, we would like to limit the false positive rate so that we ensure that the producers are not given poor predictions.


Dataset: Wine Quality URL: https://archive.ics.uci.edu/ml/datasets/wine+quality

### 2. Data Understanding

- 1.5 points: Load the dataset and appropriately define data types. What data type should be used to represent each data attribute? Discuss the attributes collected in the dataset. For datasets with a large number of attributes, only discuss a subset of relevant attributes.  
- 1.5 points: Verify data quality: Explain any missing values or duplicate data. Visualize entries that are missing/complete for different attributes. Are those mistakes? Why do these quality issues exist in the data? How do you deal with these problems? Give justifications for your methods (elimination or imputation).  

In [37]:
import pandas as pd
import numpy as np

# load red wine dataset into pandas and give it a color attribute (R)
df_red = pd.read_csv('./winequality-red.csv', sep=';')
df_red['color'] = 'R'

# load white wine dataset into pandas and give it a color attribute (W)
df_white = pd.read_csv('./winequality-white.csv', sep=';')
df_white['color'] = 'W'

# combine dataset into one pandas dataframe
df_full = pd.concat([df_red, df_white], axis=0, ignore_index=True)

#print number of rows in each df to confirm all data is loaded in
print('Length of red wine dataset:', df_red.shape[0])
print('Length of white wine dataset:', df_white.shape[0])
print('Length of all wine dataset:', df_full.shape[0])

df_full.head()

Length of red wine dataset: 1599
Length of white wine dataset: 4898
Length of all wine dataset: 6497


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,R
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,R
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,R
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,R
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,R


#### TODO: analysis of features

In [38]:
print(df_full.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  color                 6497 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB
None


#### TODO: analysis on non-null

In [39]:
#summarize data
df_full.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


#### TODO: analysis of count values and possible outliers (get into explination during visualization though)

In [50]:
#find all duplicated instances
duplicated_indexes = df_full.duplicated()
print('Duplicated instances:', len(df_full[duplicated_indexes]))

duplicated_reds = df_red.duplicated()
df_red[duplicated_reds]

#duplicated_whites = df_white.duplicated()
#df_white[duplicated_whites]

Duplicated instances: 1177


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
4,7.4,0.700,0.00,1.90,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,R
11,7.5,0.500,0.36,6.10,0.071,17.0,102.0,0.99780,3.35,0.80,10.5,5,R
27,7.9,0.430,0.21,1.60,0.106,10.0,37.0,0.99660,3.17,0.91,9.5,5,R
40,7.3,0.450,0.36,5.90,0.074,12.0,87.0,0.99780,3.33,0.83,10.5,5,R
65,7.2,0.725,0.05,4.65,0.086,4.0,11.0,0.99620,3.41,0.39,10.9,5,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1563,7.2,0.695,0.13,2.00,0.076,12.0,20.0,0.99546,3.29,0.54,10.1,5,R
1564,7.2,0.695,0.13,2.00,0.076,12.0,20.0,0.99546,3.29,0.54,10.1,5,R
1567,7.2,0.695,0.13,2.00,0.076,12.0,20.0,0.99546,3.29,0.54,10.1,5,R
1581,6.2,0.560,0.09,1.70,0.053,24.0,32.0,0.99402,3.54,0.60,11.3,5,R


#### Todo: analysis on dupes

In [36]:
#remove all duplicates
print('Number of instances before dropped duplicates:', df_full.shape[0])
df_full = df_full.drop_duplicates()
print('Number of instances after dropped duplicates:', df_full.shape[0])

Number of instances before dropped duplicates: 6497
Number of instances after dropped duplicates: 5320


### 3. Data Visualization

- 2 points: Visualize basic feature distributions. That is, plot the dynamic range and exploratory distribution plots (like boxplots, histograms, kernel density estimation) to better understand the data. Describe anything meaningful or potentially useful you discover from these visualizations. These may also help to understand what data is missing or needs imputation. __Note:__ You can also use data from other sources to bolster visualizations. Visualize at least five plots, at least one categorical and at least one numeric. 
- 2.5 points: Ask three interesting questions that are relevant to your dataset and explore visuals that help answer these questions. Use whichever visualization method is appropriate for your data.  __Important:__ Interpret the implications for each visualization. 

### 4. Exceptional Work

- You have free reign to provide any additional analyses. 
- One idea (required for 7000 level students): implement dimensionality reduction using uniform manifold approximation and projection (UMAP), then visualize and interpret the results. Give an explanation of UMAP dimensionality reduction methods. You may be interested in the following information: https://github.com/lmcinnes/umap 

### Reference

[1] UCI Machine Learning Repository. Wine Quality Dataset. https://archive.ics.uci.edu/ml/datasets/Wine+Quality

[2] https://rstudio-pubs-static.s3.amazonaws.com/57835_c4ace81da9dc45438ad0c286bcbb4224.html

[3] Penn State Eberly College of Science. Analysis of Wine Quality Data. https://online.stat.psu.edu/stat508/lesson/analysis-wine-quality-data
